In [33]:
#Importing important libraries
import requests
import numpy as np
import pandas as pd
from sshtunnel import SSHTunnelForwarder
import psycopg2 as psy
import time

In [34]:
#datawarehouse connection
def get_conn_postgres(SSH_required,key_path):

    db='datawarehouse'
    DB_HOST='datawarehouse.cdgpvetprks3.ap-south-1.rds.amazonaws.com'
    conn = []
    if SSH_required == 'Yes':
        SSH_HOST='ec2-15-206-161-154.ap-south-1.compute.amazonaws.com'
        #LOCALHOST="0.0.0.0"
        ssh_tunnel= SSHTunnelForwarder(
                (SSH_HOST, 22),
                ssh_username="ec2-user",
                ssh_private_key= key_path,
                ssh_private_key_password= "",
                remote_bind_address=(DB_HOST, 5432),
                local_bind_address=('127.0.0.1', 0)
        )
        print('Tunnel Started')
        ssh_tunnel.start()
        conn = psy.connect(
            host=ssh_tunnel.local_bind_host,
            port=ssh_tunnel.local_bind_port,
            user='postgres',
            password= "Simply1234",
            database='postgres')
        print('Connection Made')
        return conn
    else:
        conn = psy.connect(
            host = DB_HOST,
            port = 5432,
            user = 'postgres',
            password= "Simply1234",
            database='postgres')
        print('Connection Made')
        return conn

In [35]:
#read replica connection
def get_conn_readreplica(SSH_required,key_path):   #for getting a connection as a result

    db='datawarehouse'
    DB_HOST='gs-prod-new-db-read.cdgpvetprks3.ap-south-1.rds.amazonaws.com'
    conn = []
    if SSH_required == 'Yes':
        SSH_HOST='ec2-15-206-161-154.ap-south-1.compute.amazonaws.com'
        #LOCALHOST="0.0.0.0"
        ssh_tunnel= SSHTunnelForwarder(
                (SSH_HOST, 22),
                ssh_username="ec2-user",
                ssh_private_key= key_path,
                ssh_private_key_password= "",
                remote_bind_address=(DB_HOST, 5432),
                local_bind_address=('127.0.0.1', 0)
        )
        print('Tunnel Started')
        ssh_tunnel.start()
        conn = psy.connect(
            host=ssh_tunnel.local_bind_host,
            port=ssh_tunnel.local_bind_port,
            user='postgres',
            password= "Simply1234",
            database='postgres')
        print('Connection Made')
        return conn
    else:
        conn = psy.connect(
            host = DB_HOST,
            port = 5432,
            user = 'postgres',
            password= "Simply1234",
            database='postgres')
        print('Connection Made')
        return conn

In [36]:
#datawarehouse, for getting a datafarame as a result
def get_df_from_sql_postgres(SSH_required, query,key_path):   

    db='datawarehouse'
    DB_HOST='datawarehouse.cdgpvetprks3.ap-south-1.rds.amazonaws.com'
    conn = None
    if SSH_required == 'Yes':
        SSH_HOST='ec2-15-206-161-154.ap-south-1.compute.amazonaws.com'
        #LOCALHOST="0.0.0.0"
        ssh_tunnel= SSHTunnelForwarder(
                (SSH_HOST, 22),
                ssh_username="ec2-user",
                ssh_private_key= key_path,
                ssh_private_key_password= "",
                remote_bind_address=(DB_HOST, 5432),
                local_bind_address=('127.0.0.1', 0)
        )
        # ssh_tunnel._server_list[0].block_on_close = False
        ssh_tunnel.start()
        conn = psy.connect(
            host=ssh_tunnel.local_bind_host,
            port=ssh_tunnel.local_bind_port,
            user='postgres',
            password= "Simply1234",
            database='postgres')
        df_results = pd.read_sql(query, conn)
        conn.close()
        ssh_tunnel.stop()
        return df_results
    else:
        conn = psy.connect(
            host = DB_HOST,
            port = 5432,
            user = 'postgres',
            password= "Simply1234",
            database='postgres')
        df_results = pd.read_sql(query, conn)
        conn.close()
        return df_results

In [37]:
#read replica, for getting a datafarame as a result
def get_df_from_sql_readreplica(SSH_required, query,key_path):   #for getting a datafarame as a result

    db='datawarehouse'
    DB_HOST='gs-prod-new-db-read.cdgpvetprks3.ap-south-1.rds.amazonaws.com'
    conn = None
    if SSH_required == 'Yes':
        SSH_HOST='ec2-15-206-161-154.ap-south-1.compute.amazonaws.com'
        #LOCALHOST="0.0.0.0"
        ssh_tunnel= SSHTunnelForwarder(
                (SSH_HOST, 22),
                ssh_username="ec2-user",
                ssh_private_key= key_path,
                ssh_private_key_password= "",
                remote_bind_address=(DB_HOST, 5432),
                local_bind_address=('127.0.0.1', 0)
        )
        # ssh_tunnel._server_list[0].block_on_close = False
        ssh_tunnel.start()
        conn = psy.connect(
            host=ssh_tunnel.local_bind_host,
            port=ssh_tunnel.local_bind_port,
            user='postgres',
            password= "Simply1234",
            database='postgres')
        df_results = pd.read_sql(query, conn)
        conn.close()
        ssh_tunnel.stop()
        return df_results
    else:
        conn = psy.connect(
            host = DB_HOST,
            port = 5432,
            user = 'postgres',
            password= "Simply1234",
            database='postgres')
        df_results = pd.read_sql(query, conn)
        conn.close()
        return df_results

In [38]:
#defining function to use api
def make_request(address, city):
    url = 'http://grow-simplee-nlb-prod-a264c46571856f67.elb.ap-south-1.amazonaws.com:9016/parse_address'
    headers = {'Content-Type': 'application/json'}
    data = {
        "address": address,
        "city": city
    }
    try:
        response = requests.post(url, headers=headers, json=data)
        response.raise_for_status()  # Raise an error for bad status codes
        
        response_json = response.json()
        
        # Extract lat_lng from the response JSON
        lat_lng = response_json.get('lat_lng', None)
        return lat_lng if lat_lng else None
        
    except requests.RequestException as e:
        print(f"An error occurred: {e}")
        print(f"Response content: {response.content}")
        return None

In [39]:
# data = {
#     'address': ['HYDERABAD Flat 106 Ashoka Spring Fields Kailash Nagar Ameenpur road Chandanagar Nursery before apartments on main road'],
#     'city': ['Hyderabad']
# }
# df = pd.DataFrame(data)

# # Get latitude and longitude for each address
# df['lat_lng'] = df.apply(lambda row: make_request(row['address'], row['city']), axis=1)

# print(df)

In [70]:
#Getting Ops Main Warehouses for filtering Active Warehouses from warehouses_warehouse database
SSH_required = 'Yes'
key_path = '/Users/rajatsansaniwal/Documents/tunnel-ssh .cer'
query = "select distinct(warehouse_name) from public.ops_main where date_trunc('month', created_date) >= date_trunc('month', now() - interval'1 month') and shipping_partner = 'Hyperlocal' and shipping_city = 'Bangalore' and warehouse_city = 'Bangalore';"
# Establish a connection
conn = get_conn_postgres(SSH_required, key_path)

# Retrieve data into a DataFrame
df_ops_main = get_df_from_sql_postgres(SSH_required, query, key_path)

# Now you can perform further operations with the DataFrame
print(df_ops_main)

Tunnel Started
Connection Made
                                       warehouse_name
0                                        Designer BLR
1                         SNITCH APPARELS PVT LTD NEL
2      16   1, Deganhalli Village Road, Kasaba Hobli,
3                          Manash Bangalore Warehouse
4                                  EASYECOM-RDCBLRFC4
5   48,1st Cross, Chowdappa layout,,Opp  Federal B...
6                                      BOAT Bangalore
7                                           OZiva BLR
8                                    BLR Adret Retail
9                                                Plum
10                Indiejewel Fashions Private Limited
11                                              Jigni
12                                      Bangalore ANS
13                                  Ketch Bangalore 1
14                                      PRX_Warehouse
15                   Intellihealth solutions pvt ltd 
16                                           CURESK

In [85]:
# Usage with the actual path to the private key
SSH_required = 'Yes'
key_path = '/home/rajat/Downloads/tunnel-ssh .cer'
query = "select * from public.warehouses_warehouse"

# Establish a connection
conn = get_conn_readreplica(SSH_required, key_path)

# Retrieve data into a DataFrame
df_warehouse = get_df_from_sql_readreplica(SSH_required, query, key_path)
df_warehouse = df_warehouse.drop_duplicates(subset=['warehouse_name', 'warehouse_pincode'])
# df_warehouse = df_warehouse.iloc[:2]

# Now you can perform further operations with the DataFrame 'df_warehouse'
print(df_warehouse)

Tunnel Started
Connection Made
      warehouse_int_id                          warehouse_id  \
1891              1994  Trusource Technology Private Limited   

                            warehouse_name  warehouse_gstin warehouse_phone  \
1891  Trusource Technology Private Limited  29AAGCT1141A1ZT      8123943414   

                                      warehouse_address   warehouse_city  \
1891  Survey No. 111/3, 111/5, 111/1, 111/7, 111/2, ...  Bangalore Rural   

     warehouse_state  warehouse_pincode contact_name contact_number  wms_id  \
1891       Karnataka             562123     Rapidbox    89898989898  TPv397   

     op_owner  shipping_auto  stock_tracking  
1891   TPv397           True            True  


In [83]:
# Drop duplicates considering all columns
df_warehouse = df_warehouse.drop_duplicates()

# Now you can check the result after dropping duplicates
print(df_warehouse[df_warehouse['warehouse_name'] == 'Trusource Technology Private Limited'])


      warehouse_int_id                          warehouse_id  \
1891              1994  Trusource Technology Private Limited   
1893              1646  Trusource Technology Private Limited   

                            warehouse_name  warehouse_gstin warehouse_phone  \
1891  Trusource Technology Private Limited  29AAGCT1141A1ZT      8123943414   
1893  Trusource Technology Private Limited  29AAGCT1141A1ZT      8123943414   

                                      warehouse_address   warehouse_city  \
1891  Survey No. 111/3, 111/5, 111/1, 111/7, 111/2, ...  Bangalore Rural   
1893  Survey No. 111/3, 111/5, 111/1, 111/7, 111/2, ...  Bangalore Rural   

     warehouse_state  warehouse_pincode contact_name contact_number  wms_id  \
1891       Karnataka             562123     Rapidbox    89898989898  TPv397   
1893       Karnataka             562123   Libin Biju     8123943414     397   

     op_owner  shipping_auto  stock_tracking  
1891   TPv397           True            True  
1893    

In [72]:
df_warehouse = pd.merge(df_ops_main, df_warehouse[['warehouse_name', 'warehouse_city', 'warehouse_address', 'warehouse_pincode']], how = 'left', on = 'warehouse_name')

In [78]:
print(df_warehouse)

                                       warehouse_name   warehouse_city  \
0                                        Designer BLR        Bangalore   
1                         SNITCH APPARELS PVT LTD NEL  Bangalore Rural   
2      16   1, Deganhalli Village Road, Kasaba Hobli,  Bangalore Rural   
3                          Manash Bangalore Warehouse  Bangalore Rural   
4                                  EASYECOM-RDCBLRFC4  Bangalore Rural   
5   48,1st Cross, Chowdappa layout,,Opp  Federal B...        BANGALORE   
6                                      BOAT Bangalore  Bangalore Rural   
7                                           OZiva BLR  Bangalore Rural   
8                                    BLR Adret Retail  Bangalore Rural   
9                                                Plum  Bangalore Rural   
10                Indiejewel Fashions Private Limited        Bangalore   
11                                              Jigni        Bangalore   
12                                    

In [79]:
query = "select * from application_db.node where city_name = 'Bangalore' and node_name <> 'BLR_OFSC'"
# Establish a connection
conn = get_conn_postgres(SSH_required, key_path)

# Retrieve data into a DataFrame
df_hubs = get_df_from_sql_postgres(SSH_required, query, key_path)
# df_hubs = df_hubs.iloc[:2]

# Now you can perform further operations with the DataFrame 'df_warehouse'
print(df_hubs)

df_active_hubs = pd.read_excel('active_hubs.xlsx')
# print(df_active_hubs)

# Merge the dataframes based on the 'Node Name' column
active_hubs = pd.merge(df_hubs, df_active_hubs, left_on='node_name', right_on='Node Name', how='inner')

# Drop the 'Node Name' column as it's duplicated
active_hubs.drop(columns=['Node Name', 'Count'], inplace=True)

# Print the result
print(active_hubs)

Tunnel Started
Connection Made
    node_id  warehouse_id       node_name  location_id      team_names  \
0        18           182  UUTR-Franchise            1    BLR_FRH_UUTR   
1        31           182  STNG-Franchise            1    BLR_FRH_STNG   
2        12           182  NGWR-Franchise            1    BLR_FRH_NGWR   
3         6           146      GSCAN TEST            1    test_lm_hub1   
4         2           182     DS BLR BOMM            2          blr-lm   
5         1           182      DS BLR MTH            1          blr-lm   
6        10           182    BLR FRH ECTY            1    BLR_FRH_ECTY   
7         7           182     DS BLR HBBL            1  GS_BLR_LM_HBBL   
8        33           182  BLDR-Franchise            1    BLR_FRH_BLDR   
9        32           182  CMRJ-Franchise            1    BLR_FRH_CMRJ   
10       35           182  YLHK-Franchise            1    BLR_FRH_YLHK   

    fuel_rate  distance_limit sort_codes  \
0           0               0   BLR/

In [80]:
warehouse_lat_long_list = []

for index, row in df_warehouse.iterrows():
    # Combine warehouse_address and warehouse_pincode
    address = f"{row['warehouse_address']}, {row['warehouse_pincode']}"
    city = row['warehouse_city']
    lat_long = make_request(address, city)
    warehouse_lat_long_list.append(lat_long)
    # time.sleep(5)  # Delay between requests to avoid overloading the server

# Add the results to the DataFrame
df_warehouse['lat_long'] = warehouse_lat_long_list

# Print the DataFrame with latitude and longitude
print("\nDataFrame with lat_long:")
print(df_warehouse)


DataFrame with lat_long:
                                       warehouse_name   warehouse_city  \
0                                        Designer BLR        Bangalore   
1                         SNITCH APPARELS PVT LTD NEL  Bangalore Rural   
2      16   1, Deganhalli Village Road, Kasaba Hobli,  Bangalore Rural   
3                          Manash Bangalore Warehouse  Bangalore Rural   
4                                  EASYECOM-RDCBLRFC4  Bangalore Rural   
5   48,1st Cross, Chowdappa layout,,Opp  Federal B...        BANGALORE   
6                                      BOAT Bangalore  Bangalore Rural   
7                                           OZiva BLR  Bangalore Rural   
8                                    BLR Adret Retail  Bangalore Rural   
9                                                Plum  Bangalore Rural   
10                Indiejewel Fashions Private Limited        Bangalore   
11                                              Jigni        Bangalore   
12          

In [26]:
#Finding lat long for active hubs
hubs_lat_long_list = []

for index, row in active_hubs.iterrows():
    lat_long = make_request(row['address'], row['city_name'])
    hubs_lat_long_list.append(lat_long)
    # time.sleep(5)  # Delay between requests to avoid overloading the server

# Add the results to the DataFrame
active_hubs['lat_long'] = hubs_lat_long_list

# Print the DataFrame with latitude and longitude
print("\nDataFrame with lat_long:")
print(active_hubs)


DataFrame with lat_long:
   node_id  warehouse_id       node_name  location_id      team_names  \
0       18           182  UUTR-Franchise            1    BLR_FRH_UUTR   
1       31           182  STNG-Franchise            1    BLR_FRH_STNG   
2        2           182     DS BLR BOMM            2          blr-lm   
3        1           182      DS BLR MTH            1          blr-lm   
4       10           182    BLR FRH ECTY            1    BLR_FRH_ECTY   
5        7           182     DS BLR HBBL            1  GS_BLR_LM_HBBL   
6       33           182  BLDR-Franchise            1    BLR_FRH_BLDR   
7       32           182  CMRJ-Franchise            1    BLR_FRH_CMRJ   
8       35           182  YLHK-Franchise            1    BLR_FRH_YLHK   

   fuel_rate  distance_limit sort_codes  \
0          0               0   BLR/UTTR   
1          0               0   BLR/STNG   
2          3             100   BLR/JPNR   
3          3             100   BLR/MRTH   
4          0             100

In [81]:
# Save DataFrame to a CSV file
csv_file_path = 'warehouses_w_lat_lng_blr.csv'
df_warehouse.to_csv(csv_file_path, index=False)

# Provide a link to download the CSV file
print(f"Warehouse file saved as {csv_file_path}.")

Warehouse file saved as warehouses_w_lat_lng_blr.csv.


2024-06-19 00:27:04,099| ERROR   | Socket exception: Network is unreachable (101)


In [27]:
# Save DataFrame to a CSV file
csv_file_path = 'hubs_w_lat_lng_blr.csv'
active_hubs.to_csv(csv_file_path, index=False)

# Provide a link to download the CSV file
print(f"Hubs file saved as {csv_file_path}.")

Hubs file saved as hubs_w_lat_lng_blr.csv.
